# **Tutorial** - Dynamic Topic Modeling with Trump's Tweets
(last updated 11-09-2022)

In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Trump's Tweets have evolved over time. These topics will be visualized and thoroughly explored. 

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

In [2]:
import pandas as pd

In [53]:
data = pd.read_csv('/content/ps_cleaned_toBERT.csv')
data.head(2)

,date,text,retweetCount,likeCount,location,hashtags,year,month,day,dayofWeek,hour,sentiment,score
0,2018-10-17,happy collaborate patients patient safety cana...,0.0,1.0,"Markham, Ontario",NaN,2018,10,17,Wednesday,19,POSITIVE,0.997435
1,2018-10-17,patient safety core mission reality equipment ...,2.0,2.0,USA,"['PatientCare', 'Radiology', 'puttingpatientsf...",2018,10,17,Wednesday,19,NEGATIVE,0.993858


In [18]:
type(data)

pandas.core.frame.DataFrame

In [54]:
data.text = data.text.astype("str")

In [55]:
data.text = data.text.replace(" amp ", "")

In [56]:
data.text.replace(" amp ", "", inplace=True)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [4]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this tutorial, we will be needing to extract all data's Tweet from his @realDonaldata account. We will be removing all retweet and focus on his original tweets. 

Moreover, since we are looking at his tweets over time, we will be saving all timestamps related to his tweets.

In [57]:
import re
import pandas as pd
from datetime import datetime

# Load data
# trump = pd.read_csv('https://drive.google.com/uc?export=download&id=1xRKHaP-QwACMydlDnyFPEaFdtskJuBa6')

# Filter
data.text = data.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
data.text = data.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
data.text = data.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
# data = data.loc[(data.retweetCount == 0) & (data.text != ""), :]
timestamps = data.date.to_list()
tweets = data.text.to_list()

In [ ]:
tweets[0]

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [58]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets)

Batches:   0%|          | 0/2727 [00:00<?, ?it/s]

2022-12-01 21:48:06,470 - BERTopic - Transformed documents to Embeddings
2022-12-01 21:49:58,738 - BERTopic - Reduced dimensionality
2022-12-01 21:50:08,817 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [59]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,38354,-1_patient_safety_care_patients
1,0,5966,0_great_forward_thanks_thank
2,1,2248,1_nurses_strike_pay_nursing
3,2,1203,2_learning_education_students_training
4,3,1115,3_maternity_maternal_baby_babies
5,4,1060,4_conference_keynote_congress_speakers
6,5,980,5_culture_blame_cultural_cultures
7,6,910,6_canadian_week_canada_canadians
8,7,793,7_pharmacy_pharmacists_pharmacist_pharmacies
9,8,754,8_vaccine_vaccines_vaccinated_vaccination


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [61]:
topic_model.get_topic(0)

[('great', 0.011930694900471044),
 ('forward', 0.009240617709044042),
 ('thanks', 0.00854017960562844),
 ('thank', 0.00851196531067689),
 ('amazing', 0.0076704487179329595),
 ('looking', 0.007258932067771023),
 ('work', 0.007011208532493086),
 ('today', 0.00692587915123658),
 ('team', 0.00683192204801893),
 ('fantastic', 0.006448480840778078)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [62]:
fig = topic_model.visualize_topics(); fig

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [63]:
topics_over_time = topic_model.topics_over_time(docs=tweets, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

8it [00:09,  1.22s/it]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [64]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [66]:
topic_model.reduce_topics(tweets, nr_topics=10)

2022-12-01 21:54:00,803 - BERTopic - Reduced number of topics from 249 to 11


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

In [68]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,66127,-1_patient_safety_care_patients
1,0,7162,0_great_team_work_thank
2,1,3306,1_nurses_strike_pay_patient
3,2,1833,2_simulation_learning_safety_patient
4,3,1636,3_conference_safety_patient_nursing
5,4,1394,4_nhs_safety_patient_pressure
6,5,1178,5_award_awards_face_congratulations
7,6,1177,6_canadian_week_safety_patient
8,7,1158,7_medical_devices_device_cybersecurity
9,8,1155,8_culture_safety_patient_ahrq


In [69]:
topic_model.visualize_topics(topic_model.get_topic_info()[:20].Topic.values, height=550, width=650)

In [70]:
topic_model.visualize_barchart(top_n_topics=10)

In [71]:
similar_topics, similarity = topic_model.find_topics("vaccine", top_n=5); similar_topics

[7, -1, 4, 3, 2]